In [1]:
import pandas as pd
import chart_studio.plotly as py
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import os
import matplotlib.pyplot as plt
from ast import literal_eval
from collections import OrderedDict
from numpy import array
from src.utils.utils_funcs import load_init_states

In [5]:
BASE_PATH = "/Users/aalbustami/Ali/PID-Tuning-for-Motion-Optimization"

de_dir = os.path.join(BASE_PATH, "DE-results", "config_1", "trial_1")

de_dfs = []
de_dfs_names = []
de_metadata = []
de_metadata_names = []
for file_ in sorted(os.listdir(de_dir)):
    if file_.endswith(".csv"):
        de_dfs.append(pd.read_csv(os.path.join(de_dir, file_)))
        de_dfs_names.append(file_)
    
    if file_.endswith(".txt"):
        with open(os.path.join(de_dir, file_), "r") as f:
            de_metadata.append(eval(f.read()))
            de_metadata_names.append(file_)

In [6]:
print(de_dfs_names)
print(de_metadata_names)

['2024-08-03-15-54-45_init_0_de.csv', '2024-08-03-15-57-58_init_1_de.csv', '2024-08-03-16-02-28_init_2_de.csv', '2024-08-03-16-57-02_init_3_de.csv', '2024-08-03-17-01-36_init_4_de.csv', '2024-08-03-17-17-05_init_5_de.csv']
['2024-08-03-15-57-14_init_0_de.txt', '2024-08-03-16-00-27_init_1_de.txt', '2024-08-03-16-04-56_init_2_de.txt', '2024-08-03-17-00-44_init_3_de.txt', '2024-08-03-17-08-39_init_4_de.txt', '2024-08-03-17-39-51_init_5_de.txt']


In [7]:
best_rows_in_each_df = pd.DataFrame(columns=de_dfs[0].columns+["init_state"])
for j, (metadata, df) in enumerate(zip(de_metadata, de_dfs)):
    df = df.sort_values(by=["settling_time"], ascending=[False])
    df = df.reset_index(drop=True)
    # print(df)
    # break
    fig = go.Figure()
    colors = np.linspace(0, 1, len(df))
    rgb_colors = [(50, 205, 50, c) for c in colors]
    rgb_colors[-1] = (0, 0, 255, 1)
    rgb_color_strings = [f"rgba{color}" for color in rgb_colors]

    min_thickness = 1
    max_thickness = 5
    thicknesses = np.linspace(min_thickness, max_thickness, len(df))


    for i, row in df.iterrows():
        # print(row['settling_time'])
        x = list(range(len(row["angle_values"])))
        y = eval(row["angle_values"])
        fig.add_trace(
            go.Scatter(
                x=x,
                y=y,
                mode="lines",
                line=dict(color=rgb_color_strings[i], width=thicknesses[i]),
                name=f'Experiment {row["experiment_id"]}',
                showlegend=False
            )
        )
        init_state = de_metadata_names[j].split("_")[1:3]
        row["init_state"] = f"{init_state[0]}_{init_state[1]}"
        if i == len(df) - 1:
            best_rows_in_each_df = pd.concat([best_rows_in_each_df, pd.DataFrame(row).T], ignore_index=True)

    fig.update_yaxes(range=[0, 120])

    fig.add_shape(
        type="line",
        x0=0,
        x1=1,
        y0=90,
        y1=90,
        xref="paper",
        yref="y",
        line=dict(color="black", width=4, dash="dash"),
    )

    # Update layout
    fig.update_layout(
        title=f"Initial State: {int(de_dfs_names[j].split('_')[2])+1}",
        xaxis_title="Time (s * 10^-1)",
        yaxis_title="Angle Value",
        legend_title="Experiments",
        # template='plotly_dark'
    )

    fig.show()


In [8]:
fig = go.Figure()
# colors = np.linspace(0, 1, len(best_rows_in_each_df))
# rgb_colors = [(50, 205, 50, 1) for _ in colors]
# # rgb_colors[-1] = (0, 0, 255, 1)
# rgb_color_strings = [f"rgba{color}" for color in rgb_colors]

# make the colors random with the same alpha value
# np.random.seed(0)
# colors = np.random.rand(len(best_rows_in_each_df), 3)
colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255), (0, 255, 255)]
alphas = np.ones(len(best_rows_in_each_df))
rgb_colors = np.hstack([colors, alphas.reshape(-1, 1)])
rgb_color_strings = [f"rgba{tuple(color)}" for color in rgb_colors]

min_thickness = 3
max_thickness = 3
thicknesses = np.linspace(min_thickness, max_thickness, len(best_rows_in_each_df))


for i, row in best_rows_in_each_df.iterrows():
    # print(row['settling_time'])
    x = list(range(len(row["angle_values"])))
    y = eval(row["angle_values"])
    fig.add_trace(
        go.Scatter(
            x=x,
            y=y,
            mode="lines",
            line=dict(color=rgb_color_strings[i], width=thicknesses[i]),
            name=f'Initial State: {row["init_state"].split("_")[1]}',
        )
    )

fig.update_yaxes(range=[0, 120])

fig.add_shape(
    type="line",
    x0=0,
    x1=1,
    y0=90,
    y1=90,
    xref="paper",
    yref="y",
    line=dict(color="black", width=4, dash="dash"),
)

# Update layout
fig.update_layout(
    # title=f"DE Results",
    xaxis_title="Time (s * 10^-1)",
    yaxis_title="Angle Value",
    legend_title="Experiments",
    # template='plotly_dark'
)

In [9]:
header = [
    "init_state (Kp, Ki, Kd)",
    "#_trials",
    "settling_time (%)",
    "rise_time (ms)",
    "overshoot (%)",
    "time taken (ms)",
    "steady_state_error (%)",
    "Kp",
    "Ki",
    "Kd",
]

rows = []
init_stats = load_init_states(
    "/Users/aalbustami/Ali/PID-Tuning-for-Motion-Optimization/init_states.json"
)

for i, (metadata, df) in enumerate(zip(de_metadata, de_dfs)):
    row_content = []
    row_content.append(init_stats[i])
    row_content.append(metadata["n_trials"])
    row_content.append(metadata["settling_time"])
    row_content.append(best_rows_in_each_df.iloc[i]["rise_time"])
    row_content.append(round(best_rows_in_each_df.iloc[i]["overshoot"], 3))
    row_content.append(int(metadata["total_exp_time"]))
    row_content.append(
        np.round(
            abs(
                literal_eval(best_rows_in_each_df.iloc[i]["angle_values"])[-1]
                - 90
            )
            / 90
            * 100,
            4,
        )
    )
    row_content.append(round(metadata["x"][0], 5))
    row_content.append(round(metadata["x"][1], 5))
    row_content.append(round(metadata["x"][2], 5))
    rows.append(row_content)

fig = go.Figure(
    data=[
        go.Table(
            header=dict(
                values=header, fill_color="paleturquoise", align="left"
            ),
            cells=dict(
                values=list(zip(*rows)), fill_color="lavender", align="left"
            ),
        )
    ]
)

fig.show()